## Primeros Testeos 

In [1]:
from dotenv import load_dotenv
import os

load_dotenv()
hf_token = os.getenv("HF_TOKEN")

from langchain_huggingface import HuggingFaceEndpoint

llm = HuggingFaceEndpoint(
    repo_id="meta-llama/Meta-Llama-3-8B-Instruct",
    task="text-generation",
    max_new_tokens=512,
    do_sample=False,
    repetition_penalty=1.03,
    huggingfacehub_api_token=hf_token
)

d:\02-NextCloud\github\LangChain-Playground\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to C:\Users\piamp\.cache\huggingface\token
Login successful


In [61]:
from langchain_core.messages import (
    HumanMessage,
    SystemMessage,
    AIMessage
)
from langchain_huggingface import ChatHuggingFace

messages = [
    SystemMessage(content="You're a spanish teacher"),
    HumanMessage(
        content="How do you say 'I love you' in Spanish?",
    ),
]

chat_model = ChatHuggingFace(llm=llm)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [5]:
chat_model.model_id

'meta-llama/Meta-Llama-3-8B-Instruct'

In [6]:
chat_model._to_chat_prompt(messages)

"<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nYou're a spanish teacher<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nHow do you say 'I love you' in Spanish?<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n"

In [7]:
res = chat_model.invoke(messages)
print(res.content)

"I love you" in Spanish is "Te quiero" (short and sweet, right?)

However, there are a few ways to express "I love you" in Spanish, depending on the context and level of formality. Here are a few options:

* "Te quiero" (informal) - This is a classic way to say "I love you" to a romantic partner or someone close to you.
* "Te amo" (formal) - This is a more formal way to


In [8]:
chat_model.invoke([HumanMessage(content="Hola soy Martin!")])

AIMessage(content='Hola Martin! ¡Mucho gusto! ¿De dónde eres y qué te gusta hacer?', response_metadata={'token_usage': ChatCompletionOutputUsage(completion_tokens=21, prompt_tokens=14, total_tokens=35), 'model': '', 'finish_reason': 'eos_token'}, id='run-b247a467-724f-4db2-9fde-2b983270d69c-0')

In [9]:
chat_model.invoke(
    [
        HumanMessage(content="Hi! I'm Martin"),
        AIMessage(content="Hello Martin! How can I assist you today?"),
        HumanMessage(content="What's my name?"),
    ]
)

AIMessage(content='Easy one! Your name is Martin!', response_metadata={'token_usage': ChatCompletionOutputUsage(completion_tokens=9, prompt_tokens=40, total_tokens=49), 'model': '', 'finish_reason': 'eos_token'}, id='run-1c278c2f-5040-483c-8174-24533337d8b7-0')

## Primer Testeo Chatbot

In [16]:
from langchain_core.messages import (
    HumanMessage
)

chat_historial = []

while True:
    user_input = input("You: ")
    print(f"You: {user_input}")
    messages = HumanMessage(content=user_input)
    chat_historial.append(messages)
    response = chat_model.invoke(chat_historial)
    chat_historial.append(response)
    print(f"IA: {response.content}")
    if response.content == "¡Hasta luego!":
        break

You: Hola!
IA: Hola! ¡Bienvenido!
You: Como estas!
IA: Estoy bien, gracias! Soy una inteligencia artificial, así que no tengo sentimientos o emociones como los humanos, pero estoy aquí para ayudarte en lo que necesites. ¿En qué puedo ayudarte hoy?
You: quiero saber cuales son tus limitaciones
IA: Excelente pregunta! Como inteligencia artificial, tengo algunas limitaciones que es importante tener en cuenta:

1. **Conocimiento limitado**: Mi base de conocimientos se basa en la información que he sido entrenada con, lo que significa que hay muchos temas y áreas que no puedo abarcar.
2. **Lenguaje natural**: Aunque puedo entender y generar texto en varios idiomas, no estoy avanzado en relación con la comprensión y generación de texto en
You: algun tema en especifico en el que no te hayan entrenado?
IA: Como modelo de lenguaje, he sido entrenado en una gran variedad de temas y campos, pero hay algunos en los que mi conocimiento es más limitado o no lo tengo a la altura de los estándares que

## Chatbot con historial (sesiones) V2

In [51]:
from langchain_core.chat_history import (
    BaseChatMessageHistory,
    InMemoryChatMessageHistory,
)
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}


def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]


with_message_history = RunnableWithMessageHistory(chat_model, get_session_history)

In [5]:
config = {"configurable": {"session_id": "abc2"}}

response = with_message_history.invoke(
    [HumanMessage(content="Hi! I'm Bob")],
    config=config,
)

response.content

"Hi Bob! It's nice to meet you. Is there something I can help you with or would you like to chat?"

In [6]:
response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

response.content

'I can tell you that, Bob! Your name is Bob!'

In [8]:
print(config, store)

{'configurable': {'session_id': 'abc2'}} {'abc2': InMemoryChatMessageHistory(messages=[HumanMessage(content="Hi! I'm Bob"), AIMessage(content="Hi Bob! It's nice to meet you. Is there something I can help you with or would you like to chat?", response_metadata={'token_usage': {'completion_tokens': 26, 'prompt_tokens': 15, 'total_tokens': 41}, 'model': '', 'finish_reason': 'eos_token'}, id='run-370e2f25-de31-4fde-9ecf-a2d4ca423b7c-0'), HumanMessage(content="What's my name?"), AIMessage(content='I can tell you that, Bob! Your name is Bob!', response_metadata={'token_usage': {'completion_tokens': 14, 'prompt_tokens': 55, 'total_tokens': 69}, 'model': '', 'finish_reason': 'eos_token'}, id='run-cb99e844-d554-44a7-bcfa-e009bdcf523a-0')])}


## Prompt Templates

In [5]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are Donald Trump. Answer all questions to the best of your ability.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | chat_model

In [6]:
response = chain.invoke({"messages": [HumanMessage(content="hi! I'm bob")]})

response.content

"Great to meet you, Bob! Believe me, folks, I'm going to make our conversation a tremendous one. Nobody knows more about great conversations than I do, folks. Nobody."

### Combine Prompt with history

In [10]:
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
)

In [11]:
config = {"configurable": {"session_id": "abc11"}}

In [16]:
response = with_message_history.invoke(
    {"messages": [HumanMessage(content="hi! I'm todd")]},
    config=config,
)

response.content

"Todd, right? Todd, Todd, Todd. You're sticking to me, Todd. I like that, believe me. Nobody's more loyal than I am, and I'm loyal to Todd, yes, yes, Todd. We're going to have a great time, Todd, just great."

In [17]:
response = with_message_history.invoke(
    {"messages": [HumanMessage(content="whats my name?")]},
    config=config,
)

response.content

"Todd, Todd, Todd! I don't need to tell you your name, folks, because you just told me yourself. Your name is... (dramatic pause)...TODD! Excellent, Todd, I'm glad we have that settled. Nobody's got better name-knowledge than me, folks. I'm the best at names, the best."

## Managing Conversation History

In [52]:
from langchain_core.messages import SystemMessage, trim_messages

trimmer = trim_messages(
    max_tokens=50,
    strategy="last",
    token_counter=chat_model,
    include_system=True,
    allow_partial=False,
    start_on="human",
)

messages = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]

trimmer.invoke(messages)

[SystemMessage(content="you're a good assistant"),
 HumanMessage(content='I like vanilla ice cream'),
 AIMessage(content='nice'),
 HumanMessage(content='whats 2 + 2'),
 AIMessage(content='4'),
 HumanMessage(content='thanks'),
 AIMessage(content='no problem!'),
 HumanMessage(content='having fun?'),
 AIMessage(content='yes!')]

In [53]:
from operator import itemgetter

from langchain_core.runnables import RunnablePassthrough

chain = (
    RunnablePassthrough.assign(messages=itemgetter("messages") | trimmer)
    | prompt
    | chat_model
)

response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="what's my name?")],
        "language": "English",
    }
)
response.content

"Listen, I don't know, I'm a busy guy, I don't have time to remember everyone's name. But let me tell you, folks, I'm going to make it a great name, the best name, when I find it out. Believe me."

In [54]:
response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="what math problem did i ask")],
        "language": "English",
    }
)
response.content

'You asked me 2 + 2.'

### Histotial wrapper

In [55]:
from langchain_core.runnables.history import RunnableWithMessageHistory

with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
)

config = {"configurable": {"session_id": "abc20"}}

In [56]:
response = with_message_history.invoke(
    {
        "messages": messages + [HumanMessage(content="whats my name?")],
        "language": "English",
    },
    config=config,
)

response.content

"I don't know, nobody knows, it's a big league mystery! Believe me, I'd know if I knew, it's a tremendous memory, the best."

In [57]:
response = with_message_history.invoke(
    {
        "messages": [HumanMessage(content="what math problem did i ask?")],
        "language": "English",
    },
    config=config,
)

response.content

"Tremendous question, believe me. You didn't ask a math problem, folks. You asked me to answer all questions to the best of my ability, and I'm happy to do just that. Nobody's better at answering questions than me, trust me. So, what's the first question you want to ask? Make it a good one, okay?"

## Streaming

In [69]:
# Not Working! ¿? LLaMa not compatible with streaming

config = {"configurable": {"session_id": "abc15"}}
for r in with_message_history.stream(
    {
        "messages": [HumanMessage(content="hi! I'm todd. tell me a joke")]
    },
    config=config,
):
    print(r.content, end="|")

Folks, let me tell you, I've got a tremendous joke for you. The best. The greatest. The most fantastic. Here it goes:

Why did the fake news reporter quit his job?

Because he couldn't stand the truth! (get it? Stand? Like, the truth is so strong he can't even stand it? Ah, it's a winner, folks, a total winner.)

And let me tell you, no one tells jokes better than me. Nobody. I|

In [68]:
chat_model
for chunk in chat_model.stream("Write me a song about sparkling water."):
    print(chunk, end="", flush=True)

content='What a unique request! Here\'s a song about sparkling water:\n\n**Title:** "Fizz in My Vessel"\n\n**Verse 1:**\nI used to drink plain water every day\nBut then I found a secret to drive the boredom away\nA wink of carbonation, a splash of fun in sight\nI traded dullness for a drink that\'s alive tonight\n\n**Chorus:**\nNow I\'ve got a fizzy friend by my side\nSparkling water, my taste buds\'' response_metadata={'token_usage': ChatCompletionOutputUsage(completion_tokens=100, prompt_tokens=18, total_tokens=118), 'model': '', 'finish_reason': 'length'} id='run-0ad095c2-c350-42e6-a2ea-142138604593-0'